## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
	-	Run:
		```bash
		cd .\02-experiment-tracking\mlflow-scenarios\
		mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local
		```
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

---

`mlflow ui`: Just a **local viewer** (communicates with local files or the backend DB directly, not via HTTP APIs). No REST API = no remote logging.

`mlflow server`: Full **tracking server**, **with REST API** (suitable for real ML workflows), letting external tools/scripts do things like:

- `POST /api/2.0/mlflow/runs/log-metric` → Log metrics

- `GET /api/2.0/mlflow/experiments/list` → List experiments

- `DELETE /api/2.0/mlflow/runs/delete` → Delete a run

- etc.


In [ ]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

In [ ]:
# sends an API request to http://127.0.0.1:5000/api/2.0/mlflow/experiments/search 
mlflow.search_experiments()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

In [ ]:
mlflow.search_experiments()

### Interacting with the model registry

In [ ]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [ ]:
client.search_registered_models()

In [ ]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)